# Prep OPERA RTC CalVal data: stage 1, part 2

**Alex Lewandowski; Alaska Satellite Facility, University of Alaska Fairbanks**

## Performs additional data preparation for OPERA RTC calibration and validation

**Notebook Requires**
- directory of geotiffs processed with Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb

**Actions**
- convert incidence angle maps from radians to degrees
- Use the Copernicus global land cover data and HyP3 layover-shadow mask to create a mask of valid land cover pixels unaffected by layover or shadow
- Determine _foreslope_, _backslope_, and _flat_ areas by subtracting the ellipsoidal incidence angles from local incidence angles
- create 3 backscatter geotiffs for each polarity containing valid pixels of the selected ground cover classification in:
    - foreslope regions
    - backslope regions
    - flat regions

In [1]:
from ipyfilechooser import FileChooser
import math
import numpy as np
from pathlib import Path
import shutil

import numpy.ma as ma
from osgeo import gdal

import opensarlab_lib as osl

In [2]:
print("Select the *_prepped directory output by Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb")
fc = FileChooser(Path.cwd())
display(fc)

Select the *_prepped directory output by Prep_OPERA_RTC_CalVal_data_stage1_part1.ipynb


FileChooser(path='/home/jovyan/calval-RTC/regression_analysis/data_prep', filename='', title='', show_hidden=F…

## Warp all products to pixel size of 30

In [3]:
data_dir = Path(fc.selected_path)
tiffs = list(data_dir.glob('*.tif'))
resolution = 30
for t in tiffs:
    gdal.Warp(str(t), str(t), xRes=resolution, yRes=resolution, 
              targetAlignedPixels=True, dstNodata=None, copyMetadata=True)   

## Glob product paths

In [4]:
local_inc = list(data_dir.glob('*_lc_inc_map_*.tif'))[0]
ellipse_inc = list(data_dir.glob('*_ell_inc_map_*.tif'))[0]
ls_map = list(data_dir.glob('*_ls_map_*.tif'))[0]
vh = list(data_dir.glob("*_VH_*.tif"))[0]
vv = list(data_dir.glob("*_VV_*.tif"))[0]
dem = list(data_dir.glob("*_dem_*.tif"))[0]
landcover = list(data_dir.glob("*LC100_global*.tif"))[0]

---
## Mask unwanted land cover types

### Landcover Classifications
https://lcviewer.vito.be/download

- Tree cover
    - 111
    - 112
    - 114
    - 115
    - 116

## Select a ground cover type of interest

In [5]:
print("Select a ground cover type")
ground_cover_choice = osl.select_parameter(["Tree_Cover"])
display(ground_cover_choice)

Select a ground cover type


RadioButtons(layout=Layout(min_width='800px'), options=('Tree_Cover',), value='Tree_Cover')

In [6]:
ground_cover = ground_cover_choice.value
ground_covers = [0, 111, 112, 113, 114, 115, 116, 121, 122, 123, 124, 125, 126, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200]

if ground_cover == 'Tree_Cover':
    valid_covers = [111 , 112, 114, 115, 116]

invalid_covers = [c for c in ground_covers if c not in valid_covers]

ground_cover_dir = data_dir.parent/ground_cover
if not ground_cover_dir.is_dir():
    ground_cover_dir.mkdir()

## Convert incidence angle maps to degrees

In [7]:
to_deg = [local_inc, ellipse_inc]
for i, ds in enumerate(to_deg):
    out = ground_cover_dir/f"{ds.stem}_deg.tif"    
    shutil.copy(ds, out)
    f = gdal.Open(str(out), gdal.GA_Update)
    array = f.ReadAsArray()

    array = array / math.pi * 180

    f.GetRasterBand(1).WriteArray(array)
    f.FlushCache()
    if i == 0:
        local_inc = out
    else:
        elipse_inc = out

## mask invalid pixels (as determined from layover-shadow mask)

## Create 3 backscatter tiffs for each polarization:

- foreslope backscatter
- backslope backscatter
- flat backscatter 

Remove invalid ground cover and layover/shadow pixels from all output backscatter tiffs 

In [ ]:
valid_landcover = ground_cover_dir/f"{landcover.stem}_valid_{ground_cover}.tif"
shutil.copy(landcover, valid_landcover)

f_landcover = gdal.Open(str(valid_landcover), gdal.GA_Update)
landcover_ar = f_landcover.ReadAsArray()

# Set all invalid ground cover pixels to 0
for val in invalid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=0)
    # mask = ma.masked_where(landcover_ar==val, landcover_ar)
    # landcover_ar = mask.filled(fill_value=0)

# set all valid ground cover pixels to 1
for val in valid_covers:
    land_cover_masked = ma.masked_values(landcover_ar, val)
    landcover_ar = land_cover_masked.filled(fill_value=1)
    # mask = ma.masked_where(landcover_ar==val, landcover_ar)
    # landcover_ar = mask.filled(fill_value=1)
    

# set all pixels affected by layover or shadow to 0
f_ls_mask = gdal.Open(str(ls_map))
ls_array = f_ls_mask.ReadAsArray()
ls_mask = ma.masked_where(ls_array!=1, ls_array)
ls_array = ls_mask.filled(fill_value=0)

# Update valid land cover mask with invalid pixels from layover shadow mask
ls_mask = ma.masked_where(ls_array==0, landcover_ar)
landcover_ar = ls_mask.filled(fill_value=0)

# # Save valid land cover mask, containing values=1 for valid land cover pixels unaffected by layover and shadow
f_landcover.GetRasterBand(1).WriteArray(landcover_ar)
f_landcover.FlushCache()

# Subtract elliposidal incidence angles from local incidence angles
local = gdal.Open(str(local_inc))
local_ar = local.ReadAsArray()
ellipse = gdal.Open(str(elipse_inc))
ellipse_ar = ellipse.ReadAsArray()
ell_minus_local_inc_arr = local_ar - ellipse_ar

# Create foreslope, backslope, and flat backscatter geotiffs for each polarization
for pol in [vh, vv]:
    foreslope = ground_cover_dir/f"{pol.stem}_foreslope.tif"
    backslope = ground_cover_dir/f"{pol.stem}_backslope.tif"
    flat = ground_cover_dir/f"{pol.stem}_flat.tif"

    slopes = [foreslope, backslope, flat]
    for s in slopes:    
        shutil.copy(pol, s)
        f = gdal.Open(str(s), gdal.GA_Update)
        pol_array = f.ReadAsArray()

        flat_threshold = [-2, 2]

        if "backslope" in str(s):
            mask = ma.masked_where(ell_minus_local_inc_arr<=2, pol_array)
            # Replace values <= upper flat_threshold with 0
            masked_array = mask.filled(fill_value=np.nan)
        elif "foreslope" in str(s):
            # Replace values >= lower flat_threshold with 0
            mask = ma.masked_where(ell_minus_local_inc_arr>=-2, pol_array)
            masked_array = mask.filled(fill_value=np.nan)
        else:
            # Replace values outside of flat_threshold with 0
            mask = ma.masked_where(ell_minus_local_inc_arr>2, pol_array)
            masked_array = mask.filled(fill_value=np.nan)
            mask = ma.masked_where(ell_minus_local_inc_arr<-2, masked_array)
            masked_array = mask.filled(fill_value=np.nan)

        # Remove invalid ground cover and layover/shadow pixels
        invalid_mask = ma.masked_where(landcover_ar==0, masked_array)
        masked_array = invalid_mask.filled(fill_value=np.nan)

        f.GetRasterBand(1).WriteArray(masked_array)
        f.FlushCache()

*Prep_OPERA_RTC_CalVal_data_stage1_part2 - Version 2.0.1 - April 2023*

*Changes:*

- *swap stage1_part2 and stage1_part3 to finish preprocessing before MGRS tiling*